In [1]:
import sys
sys.path.append('../')
import os
os.chdir('../')
%reload_ext autoreload
%autoreload 2
import matplotlib.pyplot as plt
%matplotlib notebook
%matplotlib inline
os.environ["CUDA_VISIBLE_DEVICES"]="4,5,6,7"

In [2]:
import os
import json

import torch
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter

import models.vqvae as vqvae
import utils.losses as losses 
import options.option_vq as option_vq
import utils.utils_model as utils_model
from dataset import dataset_VQ, dataset_TM_eval
import utils.eval_trans as eval_trans
from options.get_eval_option import get_opt
from models.evaluator_wrapper import EvaluatorModelWrapper
import warnings
warnings.filterwarnings('ignore')
from utils.word_vectorizer import WordVectorizer
from exit.utils import generate_src_mask
import math

In [3]:
class Temp:
    def __init__(self):
        print('mock:: opt')
args = Temp()
args.dataname = args.dataset_name = 't2m'
args.mu = 0.99
w_vectorizer = WordVectorizer('./glove', 'our_vab')

if args.dataname == 'kit' : 
    dataset_opt_path = './checkpoints/kit/Comp_v6_KLD005/opt.txt'  
    args.nb_joints = 21
else :
    dataset_opt_path = './checkpoints/t2m/Comp_v6_KLD005/opt.txt'
    args.nb_joints = 22
wrapper_opt = get_opt(dataset_opt_path, torch.device('cuda'))
eval_wrapper = EvaluatorModelWrapper(wrapper_opt)

mock:: opt
Reading ./checkpoints/t2m/Comp_v6_KLD005/opt.txt
Loading Evaluation Model Wrapper (Epoch 28) Completed!!


### ViTVQGAN

In [4]:
args.batch_size = 256
args.down_t = 2
args.window_size = -1
train_loader = dataset_VQ.DATALoader(args.dataname,
                                        args.batch_size,
                                        window_size=args.window_size,
                                        unit_length=2**args.down_t)
train_loader_iter = dataset_VQ.cycle(train_loader)

100%|██████████| 23384/23384 [00:07<00:00, 2942.49it/s]

Total number of motions 23384


In [5]:
gt_motion_full, m_len = next(train_loader_iter)
gt_motion_full = gt_motion_full.float()

In [6]:
from models.vqvae_trans import Encoder, Decoder, VQVAE_Transformer, vqvae_wrapper
from models.quantize_cnn import QuantizeEMAReset, QuantizeEMAReset_mask
gt_motion_full = gt_motion_full.cuda()
m_len = m_len.cuda()

nb_code=512
code_dim=512
vqvae_trans = VQVAE_Transformer(args, nb_code=nb_code, code_dim=code_dim)
vqvae_trans = torch.nn.DataParallel(vqvae_trans)
vqvae_trans.cuda()

motion_dim = 251 if args.dataname == 'kit' else 263
encoder = Encoder(motion_dim=motion_dim)
encoder = torch.nn.DataParallel(encoder)
encoder.cuda()
decoder = Decoder(motion_dim=motion_dim)
decoder = torch.nn.DataParallel(decoder)
decoder.cuda()
""
# 3 losses:

# pred_motion, loss_commit, perplexity = vqvae_trans(gt_motion_full, m_len, quantizer, type='full')
# loss_commit = loss_commit.view(args.batch_size, -1, loss_commit.shape[-1])
# target = vqvae_trans(gt_motion_full, m_len, type='encode')
# pred_motion = vqvae_trans(target, torch.ceil(m_len/4), type='decode')
# pred_motion.shape, loss_commit.shape

''

In [7]:
patch_size = 4
motion = gt_motion_full
quantizer = QuantizeEMAReset(nb_code, code_dim, args)

x_e = encoder(motion, m_len).permute(0,2,1)
x_d, loss_commit, perplexity  = quantizer(x_e)
x_d = x_d.permute(0,2,1)
m_token_len = torch.ceil(m_len/patch_size)
pred_motion = decoder(x_d, m_token_len)
loss_commit.shape

torch.Size([12544, 512])

In [8]:
gt_motion = gt_motion_full
quantizer = QuantizeEMAReset_mask(nb_code, code_dim, args)
patch_size = 4
vqvae = vqvae_wrapper(patch_size, encoder, decoder, quantizer)
pred_motion, x_d, loss_commit, perplexity = vqvae(gt_motion, m_len)

In [9]:
loss_commit.shape

torch.Size([9387, 512])

In [10]:
args.commit = 0.02
args.loss_vel = 0.5
args.recons_loss = 'l1_smooth'
Loss = losses.ReConsLoss(args.recons_loss, args.nb_joints)



m_token_len = torch.ceil(m_len/patch_size)
max_token_len = math.ceil(gt_motion_full.shape[1]/patch_size)
seq_mask = generate_src_mask(gt_motion_full.shape[1], m_len)
seq_token_mask = generate_src_mask(max_token_len, m_token_len)

# motion loss
loss_motion = Loss(pred_motion, gt_motion_full).mean(-1)
loss_vel = Loss.forward_vel(pred_motion, gt_motion_full).mean(-1)

weights = seq_mask / (seq_mask.sum(1).unsqueeze(-1) * seq_mask.shape[0])
weights = torch.masked_select(weights, seq_mask)
loss_motion = torch.masked_select(loss_motion, seq_mask)
loss_vel = torch.masked_select(loss_vel, seq_mask)


# token loss
loss_commit = loss_commit.mean(-1)
token_weights = seq_token_mask / (seq_token_mask.sum(1).unsqueeze(-1) * seq_token_mask.shape[0])
token_weights = torch.masked_select(token_weights, seq_token_mask)
# loss_commit = torch.masked_select(loss_commit, seq_token_mask)

# all loss
loss_motion = (loss_motion*weights).sum()
loss_vel = (loss_vel*weights).sum()
loss_commit = (loss_commit * token_weights).sum()
loss = loss_motion + args.loss_vel * loss_vel + args.commit * loss_commit
# loss.backward()


torch.Size([256, 196]) torch.Size([256, 196])


In [44]:
token_weights.sum()

tensor(1., device='cuda:0')

### T2M-GPT (Conv)

In [39]:
args.batch_size = 2
args.down_t = 2
args.window_size = 64
train_loader = dataset_VQ.DATALoader(args.dataname,
                                        args.batch_size,
                                        window_size=args.window_size,
                                        unit_length=2**args.down_t)
train_loader_iter = dataset_VQ.cycle(train_loader)

100%|██████████| 23384/23384 [00:07<00:00, 3056.67it/s]


Total number of motions 20942


In [40]:
args.nb_code = 512 # 8192 # 
args.code_dim = 512 # 32 # 
args.output_emb_width = 512
args.down_t = 2
args.stride_t = 2
args.width = 512
args.depth = 3
args.dilation_growth_rate = 3
args.vq_act = 'relu'
args.vq_norm = None

args.quantizer = 'ema_reset'
net = vqvae.HumanVQVAE(args, ## use args to define different parameters in different quantizers
                       args.nb_code,
                       args.code_dim,
                       args.output_emb_width,
                       args.down_t,
                       args.stride_t,
                       args.width,
                       args.depth,
                       args.dilation_growth_rate,
                       args.vq_act,
                       args.vq_norm)
# net = torch.nn.DataParallel(net)
net.train()
net.cuda()

args.recons_loss = 'l1_smooth'
args.nb_joints
Loss = losses.ReConsLoss(args.recons_loss, args.nb_joints)

In [41]:
#### DEBUG 
# temp = torch.rand(256, 64, 251).cuda()
# pred_motion, loss_commit, perplexity = net(temp)
# pred_motion, loss_commit, perplexity = net(gt_motion)
# print(pred_motion.shape, loss_commit.shape, perplexity.shape)

# net = vqvae.HumanVQVAE(args, ## use args to define different parameters in different quantizers
#                        args.nb_code,
#                        args.code_dim,
#                        args.output_emb_width,
#                        args.down_t,
#                        args.stride_t,
#                        args.width,
#                        args.depth,
#                        args.dilation_growth_rate,
#                        args.vq_act,
#                        args.vq_norm)
# net.cuda()
# from torchsummary import summary
# summary(net.vqvae.encoder, (251, 64))

conv1 = torch.nn.Conv1d(in_channels=263, out_channels=512, 
                       kernel_size=3, 
                       stride=1, 
                       padding=1)
conv2 = torch.nn.Conv1d(in_channels=512, out_channels=512, 
                       kernel_size=3, 
                       stride=1, 
                       padding=9,
                       dilation=9)
conv3 = torch.nn.Conv1d(in_channels=512, out_channels=512, 
                       kernel_size=1, 
                       stride=1)
conv4 = torch.nn.Conv1d(in_channels=512, out_channels=512, 
                       kernel_size=4, 
                       stride=2)
x = torch.rand(2, 263, 64)
x = conv1(x)
x = conv2(x)
x = conv3(x)
x = conv4(x)
x.shape
# (conv1): Conv1d(512, 512, kernel_size=(3,), stride=(1,), padding=(9,), dilation=(9,))
# (conv2): Conv1d(512, 512, kernel_size=(1,), stride=(1,))
# (0): Conv1d(512, 512, kernel_size=(4,), stride=(2,), padding=(1,))


torch.Size([2, 512, 31])

In [42]:
args.commit = 0.02
args.loss_vel = 0.5

gt_motion = next(train_loader_iter)
gt_motion = gt_motion.cuda().float() # (bs, 64, dim)

pred_motion, loss_commit, perplexity = net(gt_motion)
loss_motion = Loss(pred_motion, gt_motion)
loss_vel = Loss.forward_vel(pred_motion, gt_motion)

# loss = loss_motion + args.commit * loss_commit + args.loss_vel * loss_vel
# gt_motion.shape

 torch.Size([2, 512, 32])
 torch.Size([2, 512, 16])
 torch.Size([2, 512, 16])


### Eval

In [11]:
val_loader = dataset_TM_eval.DATALoader(args.dataname, False,
                                        32,
                                        w_vectorizer,
                                        unit_length=2**args.down_t)

100%|██████████| 1460/1460 [00:00<00:00, 2909.04it/s]

Pointer Pointing at 0


In [12]:
class LoggerWriterMock:
    def __init__(self):
        self.info
    def info(self, *args):
        print(*args)
    def add_scalar(self, *args):
        print(*args)
logger = LoggerWriterMock()
logger.info('test')
writer = LoggerWriterMock()
writer.add_scalar('./Test/FID', 'fid', 'nb_iter')

test
./Test/FID fid nb_iter


In [24]:
args.out_dir = './'
a = eval_trans.evaluation_vqvae(
    args.out_dir, 
    val_loader, 
    vqvae, #net, 
    logger, 
    writer, 
    nb_iter = 1, 
    best_fid = 1, 
    best_iter = 1, 
    best_div = 1, 
    best_top1 = 1, 
    best_top2 = 1, 
    best_top3 = 1, 
    best_matching = 1, 
    eval_wrapper=eval_wrapper)


100%|██████████| 47/47 [00:05<00:00,  9.01it/s]


--> 	 Eva. Iter 1 :, FID. 32.5747, Diversity Real. 9.8223, Diversity. 2.8842, R_precision_real. [0.51928191 0.68949468 0.78390957], R_precision. [0.03989362 0.07047872 0.10039894], matching_score_real. 2.9016788969648646, matching_score_pred. 7.219740370486645
./Test/FID 32.57471793192871 1
./Test/Diversity 2.8841934 1
./Test/top1 0.0398936170212766 1
./Test/top2 0.07047872340425532 1
./Test/top3 0.10039893617021277 1
./Test/matching_score 7.219740370486645 1
--> --> 	 Diversity Improved from 1.00000 to 2.88419 !!!


In [ ]:
word_embeddings, pos_one_hots, caption, sent_len, motion, m_length, token, name = next(iter(val_loader))
motion.shape